### Установка и загрузка необходимых модулей

In [ ]:
!pip install faiss-cpu langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
import os
import getpass
import re
import requests
import openai
from langchain.docstore.document import Document

In [ ]:
# Получение ключа API от пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


### Создаем текстовый файл с диалогами из Гугл таблицы

In [ ]:
import pandas as pd
import requests
from io import BytesIO

# функция для загрузки таблицы по ссылке из гугл драйв
#def load_xls_pd(url: str) -> str:
def load_xls_pd(url: str, sheet_name: str = 0) -> str:

    # Extract the document ID from the URL
    match_ = re.search('/spreadsheets/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Sheets URL')
    doc_id = match_.group(1)

    if sheet_name is None:
        raise ValueError('Invalid sheet_name')

    # Download the table as pandas
    response = requests.get(f'https://docs.google.com/spreadsheets/d/{doc_id}/export?format=xlsx')
    response.raise_for_status()     #проверяет статус код ответа. Если получен ответ с кодом ошибки (4xx или 5xx), вызывается исключение HTTPError.
    #data = pd.read_excel(BytesIO(response.content), sheet_name=0)  #чтение первого листа из книги
    data = pd.read_excel(BytesIO(response.content),sheet_name = sheet_name, header=None) #чтение конкретного листа из Книги Excel

    ''' Разъяснение:
        Когда мы хотим прочитать данные Excel с помощью функции pd.read_excel(), она требует передачи ей пути к файлу или объекта, представляющего файл.
        В параметре io функции pd.read_excel() необходимо указать путь к файлу (в виде строки, содержащей путь к файлу) или объект файлового типа (такой как BufferedWriter, BufferedReader и другие).
        response.content возвращает содержимое ответа на запрос HTTP в виде байтового массива (bytes array).
        Чтобы передать эти данные в функцию pd.read_excel(), нужно создать объект файла из байтового массива.
        Для этой цели используется объект BytesIO из модуля io, который предоставляет интерфейс для работы с данными в памяти, как если бы они находились в файле.
    '''
    return data

# Шаг 1: Загрузить xlsx файл из Google Drive
google_sheet_url = "https://docs.google.com/spreadsheets/d/1uKrXGbXKhbvKjYU7Ao_XlsS0vQgOy8UaQxD-nZ3M8Sg/edit?usp=sharing"
sheet_name='dialogs_2023-09-11_20-54-18.csv'

#data = load_xls_pd(google_sheet_url)   #если в таблице только один лист
data = load_xls_pd(google_sheet_url, sheet_name)

# Шаг 2: Извлечь столбец с диалогами
text_column = data[data.columns[1]]

# Шаг 3: Заменить "operatorMessage: Здравствуйте" на "<operatorMessage: Здравствуйте>" для удобства деления на чанки в дальнейшем
text_column = text_column.str.replace("operatorMessage: Здравствуйте", "<operatorMessage: Здравствуйте>")

# Шаг 4: Сохранить в файл "Dialogs__.txt" в своей папке
file_path = "/content/kia_dialogs.txt"

with open(file_path, "w", encoding="utf-8") as f:
    for text in text_column:
        f.write(str(text) + "\n")

print("Файл kia_dialog.txt успешно сохранен!")

Файл kia_dialog.txt успешно сохранен!


### Функция для загрузки диалогов из Гугл докс

In [ ]:
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

Деление на чанки текстового файла

In [ ]:
loader = TextLoader("/content/kia_dialogs.txt")
documents = loader.load()

In [ ]:
#type(documents)

In [ ]:
text_splitter = CharacterTextSplitter(separator="<", chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

'''source_chunks = []     #разные методы деления текста
splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=0)

for chunk in splitter.split_text(documents):
    source_chunks.append(Document(page_content=chunk, metadata={}))'''

'source_chunks = []     #разные методы деления текста\nsplitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=0)\n\nfor chunk in splitter.split_text(documents):\n    source_chunks.append(Document(page_content=chunk, metadata={}))'

In [ ]:
# всего получилось чанков:
len(docs)

2334

In [ ]:
# первый чанк
print(docs)
page_content = docs[0].page_content
# длина первого чанка
print(len(page_content))

In [ ]:
# третий чанк:
page_content = docs[2].page_content
print(page_content)
# длина третьего чанка
print(len(page_content))

operatorMessage: Здравствуйте>.
Уточните, пожалуйста, какая информация Вас интересует? clientMessage: подскажите, я сейчас нахожусь в республике Казахстан, прошлое ТО проходила в России, в сервисном дилерском центре Киа, могу ли я пройти ТО в республике Казахстан также в сервисном центре?  operatorMessage: Уточните, пожалуйста, как к Вам можно обращаться? clientMessage: Мария operatorMessage: Мария, очень приятно. Пройти ТО у официального дилера за границей Вы можете (список на сайте www.kia.com). При этом у Вас должны стоять отметки в сервисной книжке и должен быть заказ- наряд. operatorMessage: При любом обращении к Дилеру на территории РФ, рекомендуем добавить эти отметки в историю ТО в базу официальных дилеров Киа на территории РФ. Список дилеров доступен на официальном сайте по ссылке: https://www.kia.ru/dealers/. operatorMessage: Уточните, пожалуйста, могу еще чем-либо Вам помочь? clientMessage: сервизная книжка при себе, а что значит должен быть заказ наряд? clientMessage: и буд

### Обращение к ChatGPT для обработки диалогов

Установка температуры и вывода ChatGPT

In [ ]:
temperature=0          # подставьте сюда необходимое значение
verbose=0              # подставьте сюда необходимое значение

In [ ]:
#функция удобочитаемого вывода
def insert_newlines(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + word + " ") > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += word + " "
    lines.append(current_line)
    return " ".join(lines)

In [ ]:
# Инструкция для GPT, которая будет подаваться в system
system_prompt = load_document_text('https://docs.google.com/document/d/1UrEzeIboxdz9rlaMcgZGi7DqNoMPvxHlgnNRQyWcBEw/edit?usp=sharing')   # подставьте сюда необходимое значение
user_prompt = load_document_text('https://docs.google.com/document/d/1VS5kGrUMKM-vc3d7oTdsrlQiLHRAmTZ8rM5Mb-MwO-8/edit?usp=sharing')   # подставьте сюда необходимое значение

### Подключение Google Disk для сохранения обработанной базы

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from IPython.display import clear_output

In [ ]:
itog = ""
#for i in range(len(docs)):
for i in range(272, 2334):
    #print(docs[i].page_content)  # Вывод текущего элемента списка
    print(i)
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"{user_prompt}\n{docs[i].page_content}"}
    ]
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=messages,
        temperature=temperature
    )
    answer = f'\n{insert_newlines(completion.choices[0].message.content)}'
    #itog += f'\n{answer}'

    # запись в файл
    with open("/content/drive/MyDrive/Stazhirovka_Kia/Dialogs/Useful_from_Dialog.txt", "a") as file:
        file.write(answer)
    clear_output()

1860


FileNotFoundError: ignored

In [ ]:
answer

'\n## Раздел "Техническая поддержка" - Клиент обращается с вопросом о работе сайта и ссылке на дилерский центр, которая не работает. - Оператор предоставляет клиенту  актуальную ссылку на сайт с контактами дилеров. - Клиент уточняет, является ли сайт мошенническим. - Оператор подтверждает, что ссылка активна и информация указана  корректно. - Клиент продолжает сомневаться и указывает, что при переходе по ссылке на сайт дилера в Сочи пишет, что сайт не активен. - Оператор предоставляет контактные  данные дилерского центра в Сочи. - Клиент уточняет, что происходит с сайтом. - Оператор предоставляет клиенту ссылку на официальный сайт компании Киа. - Клиент  подтверждает, что сайт заработал и благодарит оператора. - Оператор уточняет, нуждается ли клиент в дополнительной помощи. - Клиент отвечает, что нет и завершает диалог.  ## Раздел "Техническая поддержка" '

## Взаимодействие с векторной базой

### Перевод в эмбеддинги и FAISS

In [ ]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(docs, embeddings)

### Сохранение и загрузка векторной базы из диалогов локально.

In [ ]:
db.save_local("faiss_index")    #Сохранение локальной векторной базы
new_db = FAISS.load_local("faiss_index", embeddings)   #Загрузка локальной векторной базы